# Tutorial 4: Eco-Geomorphological simulation with Bathtub model
From Tutorial 2, 3 and 4 we saw how to run the hydrodynamic model, than couple it to the sediment transport model, and last how to run an EGM simulation with the bathtub approach. Here we will use the Hydrodynamic and Sediment Transport (HST) models to replace the bathtub approach.

## Dependencies
* You will need all files created with `Tutorial 1 - Creating a new domain` and `example_domain2`.
* CSV files with water levels and sediment concentration input series generated with `Tutorial 1 - Creating a new domain`

***
# STEP 0: Preparing files and loading the egmlib library
Create a folder named **tutorial05a** and copy all files from **domain01** to this new folder. Then create another folder, **tutorial05b** and copy all files from **domain02** here. Also, copy all content from **Common Files** to both new folders. Thus, we will have in folder 'a' the larger domain, with inner channel and embankment, while at folder 'b' we will have the inputs for the single-row domain.


In the code below replace the string `lib_folder` with the path to the folder where the **egmlib.py** file is and then run the cell. Just note that we should use `/` or `\\` instead of a single `\` as Python understand this last as an indicator of a special character.


The egmlib library already includes the import of many other libraries, as **numpy** and **matplotlib** that will be used later in this notebook.

In [ ]:
lib_folder = 'C:/Wetlands/Tutorials'
from sys import path
path.append(lib_folder)
from egmlib import *

In steps 1 and 2 below we will prepare a EGM simulation to run in the **tutorial05a** domain. Than in step 3, we will do the same for the domain in **tutorial05b**
***


# STEP 1: Initialising the EGM Framework (1st domain)
Let's dump the settings of the HST simulation into a `json` file. These settings are quite similar to those ones created for `Tutorial 4 - Bathtub EGM Simulation`. However, we will add a few more entries this time:
- STM parameters $Ucd$ and $ws$;
- Time-step (delta time, s) in the Hydrodynamic output files (also applied in the Sediment Transpor model);
- Initial time length (s) to be stripped off from the calculation of tidal indexes (to create an warm-up like period).


In [ ]:
# Directory's path where the model will run (1st domain)
run_folder = 'C:\\Wetlands\\Tutorials\\tutorial05a'

# Creating dictionary with information for EGM simulation
settings = {
    'title': 'HST - Large Domain',
    'Dt_EGM': 20,
    'start':'2000',
    'end':'2100',
    'roughness': {
        0: 0.12,
        1: 0.12,
        2: 0.15,
        3: 0.50
    },
    'Dcoef': 8.0,
    'input_levels_file': 'C:\\Wetlands\\Tutorials\\water_levels_1.csv',
    'input_sediment_file': 'C:\\Wetlands\\Tutorials\\sediment_conc_1.csv',
    'first_column_as_index': 'yes',
    'Ucd': 0.02,
    'ws': 0.0002,
    'Dt_HST': 300,
    'hfd_start': 48*3600
}

# Saving settings in a json file
filename = os.path.join(run_folder, 'egm_setup.json')
with open(filename, 'w') as file:
    json.dump(settings, file)

# Creating a new instance of EGMframework
egm = EGMframework()
egm.initialize(filename, run_folder)

# Printing some attributes of egm
print('\n\nNumber of cells and links:', egm.NoC, egm.NoL)
print('Water depths boundary condition applied to:\n', egm.loc_HT)
print('Series for EGM simulation:')
for i in range(egm.Nstages):
    print('    series %i: %s' % (i, egm.stages[i]))
print('Time step between stages: %i years' % egm.Dt_EGM)
print('Rule for EGM models: %s' % egm.rule)    #should return the default option: 'SE Australia'

# Save the 'periods' attribute in a file to check if hfd_start was correctly applied
periods_write(egm.periods, 'periods.csv', egm.stages)

# STEP 2: Running the 1$^{st}$ HST-EGM Simulation
With the inner methods of **EGMframework** is easy to set up a HST-EGM simulation. Within a loop through the EGM stages we need to:
1. Initialise Hydrodynamic input files
2. Run hydrodynamic model
3. Run sediment transport model
4. Run EGM models

and then, call the *finalize()* method to save the results in a binary file.


However, as we are not running the HST models within a warm-up loop, it is recommend to run the models twice for the first stage. This because the initial parameters use a single roughness throughout the domain surface, therefore not representing any particular vegetation coverage.

In [ ]:
t0 = datetime.now()    #This simulation will take about half an hour
for stix, stage in enumerate(egm.stages):
    
    print ('\nProcessing stage:', stage)
    
    if stix == 0:
        #Apply the data set from input-series files to the hydrodynamic files
        egm.initialize_hydrodynamic(stage)
        
        #Run the Hydrodynamic and Sediment Transport models
        data = egm.update_hydrodynamic(returnType='highFreqData')
        data = egm.update_sediment_transport(stage, hfd=data)
        next_ssc0 = data.ssc[-1,:]
    
    else:
        #Update parameters
        egm.initialize_hydrodynamic(stage, veg_stix=(stix-1), hfd=data)
    
    #Run the Hydrodynamic model
    data = egm.update_hydrodynamic(returnType='highFreqData')
    
    # Run the Sediment Transport model
    data.ssc0 = next_ssc0
    data = egm.update_sediment_transport(stage, hfd=data)
    next_ssc0 = data.ssc[-1,:]
    
    #Compute Hydroperiod and Mean-Depth-Below-High-Tide
    egm.H[stix,:], egm.D[stix,:] = egm.update_tidal_indexes(stix, data.h)
    
    #Define vegetation cover
    egm.V[stix,:] = egm.update_vegetation(stix)
    
    #Compute the average sediment concentration (using a depth-weighted average)
    egm.C[stix,:] = np.average(data.ssc, axis=0, weights=data.h)
    
    #Compute accretion (it will be automatically scaled up to self.Dt_EGM years) and add it to the surface
    egm.A[stix,:] = egm.update_accretion(stix)
    egm.z, egm.chndep = add_accretion(egm.z, egm.chndep, egm.A[stix,:])
    
    #Apply Hillslope Diffusion model (egm.EG = elevation gain ==> egm.z(now) - egm.z(at start))ls
    egm.EG[stix,:] = egm.update_diffusion()
    
    #Save the current stage index in an attribute to pick from this point on if the simulation crashes
    egm.current = stix

print('\nElapsed time:', (datetime.now() - t0))
egm.finalize()    #'.finalize()' creates a pickle file. But '.finalize("filename.csv")' creates a CSV one

# STEP 3: Running the 2$^{nd}$ HST-EGM Simulation
Now let's move to the second domain, create the settings file and run it

In [ ]:
# Directory's path where the model will run (1st domain)
run_folder2 = 'C:\\Wetlands\\Tutorials\\tutorial05b'

# Saving settings in a json file
settings['title'] = 'HST - Single Row'
filename = os.path.join(run_folder2, 'egm_setup.json')
with open(filename, 'w') as file:
    json.dump(settings, file)    #same settings dictionary used before

# Creating a new instance of EGMframework
egm = EGMframework()
egm.initialize(filename, run_folder2)

# Printing some attributes of egm
print('\n\nNumber of cells and links:', egm.NoC, egm.NoL)
print('Water depths boundary condition applied to:\n', egm.loc_HT)

#Running simulation. It will take about 6 minutes
t0 = datetime.now()
for stix, stage in enumerate(egm.stages):
    
    print ('\nProcessing stage:', stage)
    
    if stix == 0:
        egm.initialize_hydrodynamic(stage)
        data = egm.update_hydrodynamic(returnType='highFreqData')
        data = egm.update_sediment_transport(stage, hfd=data)
        next_ssc0 = data.ssc[-1,:]
    
    else:
        egm.initialize_hydrodynamic(stage, veg_stix=(stix-1), hfd=data)
    
    data = egm.update_hydrodynamic(returnType='highFreqData')
    data.ssc0 = next_ssc0
    data = egm.update_sediment_transport(stage, hfd=data)
    next_ssc0 = data.ssc[-1,:]
    egm.H[stix,:], egm.D[stix,:] = egm.update_tidal_indexes(stix, data.h)
    egm.V[stix,:] = egm.update_vegetation(stix)
    egm.C[stix,:] = np.average(data.ssc, axis=0, weights=data.h)
    egm.A[stix,:] = egm.update_accretion(stix)
    egm.z, egm.chndep = add_accretion(egm.z, egm.chndep, egm.A[stix,:])
    egm.EG[stix,:] = egm.update_diffusion()
    egm.current = stix

print('\nElapsed time:', (datetime.now() - t0))
egm.finalize()

# STEP 4: Plotting profiles from different simulations
Usually are not only interested in the outcomes of a single simulation, but to compare multiple scenarios. Therefore we will plot some results for different simulations and profiles:
- Bathtub simulation (from Tutorial 4)
- Single-row simulation
- Row adjacent to the inner channel in the HST - large domain simulation
- Row far away to the inner channel in the HST - large domain simulation


In the code below replace the path assigned to *tut_folder* with the directory's path where you placed the tutorial files. Then, let's load the results from each simulation in different dictionaries.

In [ ]:
tut_folder = 'C://Wetlands//Tutorials//'

# Results from Bathtub Simulation
filename = os.path.join(tut_folder, 'tutorial04//framework_data.pickle')
btub = load_to_plot(filename)

# Results from HST - Single-row Domain
filename = os.path.join(tut_folder, 'tutorial05b//framework_data.pickle')
sing = load_to_plot(filename)

# Results from HST - Large Domain
filename = os.path.join(tut_folder, 'tutorial05a//framework_data.pickle')
larg = load_to_plot(filename)

# 4.1 Hydroperiod, High Tide Depth, Average SSC
*Press SHIFT+o to toggle scrolling of output area*

In [ ]:
for i, stage in enumerate(btub['stages']):
    
    fig, (ax1, ax2, ax3) = pyplot.subplots(ncols=3, dpi=100, figsize=(12,4), constrained_layout=True)
    
    #Hydroperiod
    ax1.plot(btub['xc'][0,:], btub['H'][i,0,:], color='black', label='Bathtub')
    ax1.plot(sing['xc'][0,:], sing['H'][i,0,:], label='HST Single-Row')
    ax1.plot(larg['xc'][0,:], larg['H'][i,0,:], label='HST Plain') #first row; far from inner chanel
    ax1.plot(larg['xc'][-2,:], larg['H'][i,-2,:], label='HST Margin') #second-last row; inner chanel margin
    ax1.set_xlim(0,800)
    ax1.set_xlabel('Distance from tide input creek (m)')
    ax1.set_ylim(-1,101)
    ax1.set_ylabel('Hydroperiod (%)')
    ax1.grid()
    #ax1.legend(loc='best')
    
    #Mean Depth Below High Tide
    ax2.plot(btub['xc'][0,:], btub['D'][i,0,:], color='black', label='Bathtub')
    ax2.plot(sing['xc'][0,:], sing['D'][i,0,:], label='HST Single-Row')
    ax2.plot(larg['xc'][0,:], larg['D'][i,0,:], label='HST Plain') #first row; far from inner chanel
    ax2.plot(larg['xc'][-2,:], larg['D'][i,-2,:], label='HST Margin') #second-last row; inner chanel margin
    ax2.set_xlim(0,800)
    ax2.set_xlabel('Distance from tide input creek (m)')
    ax2.set_ylim(-0.01,1.21)
    ax2.set_ylabel('Mean Depth Below High Tide (m)')
    ax2.grid()
    ax2.legend(loc='best')
    
    #Hydroperiod
    ax3.plot(btub['xc'][0,:], btub['C'][i,0,:], color='black', label='Bathtub')
    ax3.plot(sing['xc'][0,:], sing['C'][i,0,:], label='HST Single-Row')
    ax3.plot(larg['xc'][0,:], larg['C'][i,0,:], label='HST Plain') #first row; far from inner chanel
    ax3.plot(larg['xc'][-2,:], larg['C'][i,-2,:], label='HST Margin') #second-last row; inner chanel margin
    ax3.set_xlim(0,800)
    ax3.set_xlabel('Distance from tide input creek (m)')
    ax3.set_ylim(-0.1,35.1)
    ax3.set_ylabel('Average Sediment Concentration (g/m3)')
    ax3.grid()
    #ax3.legend(loc='best')
    
    fig.suptitle(stage)
    pyplot.show()
    pyplot.close(fig)

# 4.2 Accretion and Elevation Gain

In [ ]:
for i, stage in enumerate(btub['stages']):
    
    fig, (ax1, ax2) = pyplot.subplots(ncols=2, dpi=100, figsize=(8,4), constrained_layout=True)
    
    #Accumulated Accretion
    ax1.plot(btub['xc'][0,:], btub['acA'][i,0,:], color='black', label='Bathtub')
    ax1.plot(sing['xc'][0,:], sing['acA'][i,0,:], label='HST Single-Row')
    ax1.plot(larg['xc'][0,:], larg['acA'][i,0,:], label='HST Plain') #first row; far from inner chanel
    ax1.plot(larg['xc'][-2,:], larg['acA'][i,-2,:], label='HST Margin') #second-last row; inner chanel margin
    ax1.set_xlim(0,800)
    ax1.set_xlabel('Distance from tide input creek (m)')
    ax1.set_ylim(-0.01,0.31)
    ax1.set_ylabel('Accumulated Accretion (m)')
    ax1.grid()
    
    #Elevation Gain
    ax2.plot(btub['xc'][0,:], btub['EG'][i,0,:], color='black', label='Bathtub')
    ax2.plot(sing['xc'][0,:], sing['EG'][i,0,:], label='HST Single-Row')
    ax2.plot(larg['xc'][0,:], larg['EG'][i,0,:], label='HST Plain') #first row; far from inner chanel
    ax2.plot(larg['xc'][-2,:], larg['EG'][i,-2,:], label='HST Margin') #second-last row; inner chanel margin
    ax2.set_xlim(0,800)
    ax2.set_xlabel('Distance from tide input creek (m)')
    ax2.set_ylim(-0.01,0.31)
    ax2.set_ylabel('Elevation Gain (m)')
    ax2.grid()
    ax2.legend(loc='best')
    
    fig.suptitle(stage)
    pyplot.show()
    pyplot.close(fig)

## 4.3 Vegetation maps

In [ ]:
for i, stage in enumerate(btub['stages']):
    
    fig, (ax1, ax2, ax3) = pyplot.subplots(ncols=3, dpi=100, figsize=(12,4), constrained_layout=True)
    
    #Bathtub
    plot_vegetation(axis=ax1, xgrid=btub['xf'], ygrid=btub['yf'], veg=btub['V'][i,:,:], rule=btub['rule'])
    ax1.set_xlim(0,800)
    ax1.set_title('Bathtub')
    
    #HST Single-Row
    h = plot_vegetation(axis=ax2, xgrid=sing['xf'], ygrid=sing['yf'], veg=sing['V'][i,:,:], rule=sing['rule'])
    ax2.set_xlim(0,800)
    ax2.set_title('HST Single-Row')
    ax2.legend(handles=h, loc="upper center", bbox_to_anchor=(0.5, -0.10), ncol=2)
    
    #HST Large Domain
    plot_vegetation(axis=ax3, xgrid=larg['xf'], ygrid=larg['yf'], veg=larg['V'][i,:,:], rule=larg['rule'])
    ax3.set_xlim(0,800)
    ax3.set_title('HST Large Domain')
    
    fig.suptitle(stage)
    pyplot.show()
    pyplot.close(fig)

# The End
And so ends one more Tutorial! At this point you know absolute everything to put any simulation to run. You can be creative reorganizing the procedure to call each method to attend different demands. Also, now you have a large number of examples to plot the results.


But more is to come!